<a href="https://colab.research.google.com/github/seogitae/DeepLearning/blob/main/2_1_a_first_look_at_a_neural_network_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow
tensorflow.keras.__version__

'2.4.0'

# A first look at a neural network

This notebook contains the code samples found in Chapter 2, Section 1 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

----

We will now take a look at a first concrete example of a neural network, which makes use of the Python library Keras to learn to classify 
hand-written digits. Unless you already have experience with Keras or similar libraries, you will not understand everything about this 
first example right away. You probably haven't even installed Keras yet. Don't worry, that is perfectly fine. In the next chapter, we will 
review each element in our example and explain them in detail. So don't worry if some steps seem arbitrary or look like magic to you! 
We've got to start somewhere.

The problem we are trying to solve here is to classify grayscale images of handwritten digits (28 pixels by 28 pixels), into their 10 
categories (0 to 9). The dataset we will use is the MNIST dataset, a classic dataset in the machine learning community, which has been 
around for almost as long as the field itself and has been very intensively studied. It's a set of 60,000 training images, plus 10,000 test 
images, assembled by the National Institute of Standards and Technology (the NIST in MNIST) in the 1980s. You can think of "solving" MNIST 
as the "Hello World" of deep learning -- it's what you do to verify that your algorithms are working as expected. As you become a machine 
learning practitioner, you will see MNIST come up over and over again, in scientific papers, blog posts, and so on.

The MNIST dataset comes pre-loaded in Keras, in the form of a set of four Numpy arrays:

In [4]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


mnist 데이터를 받는 코드이다. 

인자를 튜플을 사용하여 위의 형식으로 받는 이유는 mnist?를 입력하면 나온다.

`train_images` and `train_labels` form the "training set", the data that the model will learn from. The model will then be tested on the 
"test set", `test_images` and `test_labels`. Our images are encoded as Numpy arrays, and the labels are simply an array of digits, ranging 
from 0 to 9. There is a one-to-one correspondence between the images and the labels.

Let's have a look at the training data:

퍼미션 에러가 나는 경우도 있는데, sudo 를 사용해서 노트북을 열거나, 에러가 나는 폴더(데이터 저장폴더)의 권한을 쓰기 가능하도록 변경해주어야 합니다. 

In [5]:
train_images[:2,:,:]

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=uint8)

앞에 두개의 데이터를 불러온다? 위의 두개의 사진을 불러오는 것과 같음.

In [6]:
train_images.shape

(60000, 28, 28)

28 * 28 pixel 그림.

In [7]:
len(train_labels)

60000

 정답의 길이를 보여준다.

In [8]:
train_labels 

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

실제 정답 값을 출력한다. 첫 번째 그림의 정답은 5임을 알 수 있다. 

Let's have a look at the test data:

In [9]:
test_images.shape

(10000, 28, 28)

In [10]:
len(test_labels)

10000

In [11]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

Our workflow will be as follow: first we will present our neural network with the training data, `train_images` and `train_labels`. The 
network will then learn to associate images and labels. Finally, we will ask the network to produce predictions for `test_images`, and we 
will verify if these predictions match the labels from `test_labels`.

Let's build our network -- again, remember that you aren't supposed to understand everything about this example just yet.

In [12]:
from tensorflow.keras import models
from tensorflow.keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

핵심이 되는 부분 모형을 정의하는 부분

케라스 모듈을 이용하여 수행

input and ouput(다음 단계의 input 이겠지..?)에 대해서 함수를 정의해주는 것

예를 들어, 위에서는 28 * 28의 데이터를 input으로 받아서 relu 함수를 통해 512 차원의 값을 출력해주고

앞선 512 차원의 값이 다시 input이 되어서 softmax 함수를 통해 10 차원의 값을 출력해준다..? (아래 설명에서는 각 10개의 값들은 확률이라고 한다..)   


The core building block of neural networks is the "layer", a data-processing module which you can conceive as a "filter" for data. Some 
data comes in, and comes out in a more useful form. Precisely, layers extract _representations_ out of the data fed into them -- hopefully 
representations that are more meaningful for the problem at hand. Most of deep learning really consists of chaining together simple layers 
which will implement a form of progressive "data distillation". A deep learning model is like a sieve for data processing, made of a 
succession of increasingly refined data filters -- the "layers".

Here our network consists of a sequence of two `Dense` layers, which are densely-connected (also called "fully-connected") neural layers. 
The second (and last) layer is a 10-way "softmax" layer, which means it will return an array of 10 probability scores (summing to 1). Each 
score will be the probability that the current digit image belongs to one of our 10 digit classes.

To make our network ready for training, we need to pick three more things, as part of "compilation" step:

* A loss function: the is how the network will be able to measure how good a job it is doing on its training data, and thus how it will be 
able to steer itself in the right direction.
* An optimizer: this is the mechanism through which the network will update itself based on the data it sees and its loss function.
* Metrics to monitor during training and testing. Here we will only care about accuracy (the fraction of the images that were correctly 
classified).

The exact purpose of the loss function and the optimizer will be made clear throughout the next two chapters.

In [13]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

loss와 metrics의 차이가 뭘까?

왜 loss를 minimize하면서 metrics로 Accuracy를 monitoring하는 거지?

loss function과 cost funtion은 같은 말인가?

아래의 코드는 앞서 sequential로 만든 network를 어떻게 training할 것인지를 정의하는 코드라고 함.


Before training, we will preprocess our data by reshaping it into the shape that the network expects, and scaling it so that all values are in 
the `[0, 1]` interval. Previously, our training images for instance were stored in an array of shape `(60000, 28, 28)` of type `uint8` with 
values in the `[0, 255]` interval. We transform it into a `float32` array of shape `(60000, 28 * 28)` with values between 0 and 1.

In [14]:
import numpy as np
np.max(train_images)

255

In [15]:
train_images.shape

(60000, 28, 28)

In [16]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

mnist data의 pixel range는 0부터 255까지의 값을 가짐.

그런데, 딥러닝 모델은 input값이 0~1 사이의 값을 가질 때, 더 효율적!?으로 학습한다고 함.  

그래서 위의 코드에서 보는 것처럼 픽셀 값을 255로 나누어 줌. (이러한 scaling은 반드시 수행해야 하는 과정은 아니라고 함.)

현재 설명부에 해당하는 블럭의 아래, 위 코드를 보면, max값이 255에서 1로 나타나는 것을 알 수 있음.


그리고 지금 reshape을 통해서 train data의 dimension을 바꿔주는 것을 알 수 있음. 

이 또한, 현재의 블럭 아래, 위 코드를 보면 (60000, 28, 28) -> (60000. 784)를 통해 확인할 수 있음. 

앞에서 모델 construct한 것을 보면, 첫번째 layer에서 input data의 shape을 28 * 28로 받는 것을 알 수 있음. (같이 확인할 것.)

In [17]:
import numpy as np
np.max(train_images)

1.0

In [18]:
train_images.shape

(60000, 784)

We also need to categorically encode the labels, a step which we explain in chapter 3:

In [19]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [20]:
from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

label 값을 one-hot-encoding을 수행하여 바꿔주기 위해 수행하는 코드. 

이 블럭의 위쪽 블럭을 보면 정답값이 0, 5, 4 처럼 정수형으로 되어있음.

그러나 현재의 블럭의 코드를 수행하고 나면, 이 블럭의 아래 블럭을 보면 정답값의 표현이 one-hot-encoding되어 있음을 알 수 있음. 

In [21]:
train_labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

label 값을 one-hot-encoding을 수행하여 바꿔준 결과

We are now ready to train our network, which in Keras is done via a call to the `fit` method of the network: 
we "fit" the model to its training data.

In [22]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 6s 10ms/step - loss: 0.4245 - accuracy: 0.8782
Epoch 2/5
469/469 [==============================] - 5s 10ms/step - loss: 0.1077 - accuracy: 0.9682
Epoch 3/5
469/469 [==============================] - 5s 12ms/step - loss: 0.0653 - accuracy: 0.9803
Epoch 4/5
469/469 [==============================] - 6s 12ms/step - loss: 0.0479 - accuracy: 0.9858
Epoch 5/5
469/469 [==============================] - 5s 10ms/step - loss: 0.0352 - accuracy: 0.9904


웨이트를 업데이트 하는 코드이다

배치 사이즈 : 샘플을 몇 개씩 사용할 것인가? 128개이니까 한 걸음에 128개씩 사용하면서 나가겠다는 의미이다. 

에       폭 : 하나의 트레인 데이터셋을 몇 번 재사용 할 것이냐의 의미이다. 

Two quantities are being displayed during training: the "loss" of the network over the training data, and the accuracy of the network over 
the training data.

We quickly reach an accuracy of 0.989 (i.e. 98.9%) on the training data. Now let's check that our model performs well on the test set too:

In [23]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.0652 - accuracy: 0.9793


In [24]:
print('test_acc:', test_acc)

test_acc: 0.9793000221252441



Our test set accuracy turns out to be 97.8% -- that's quite a bit lower than the training set accuracy. 
This gap between training accuracy and test accuracy is an example of "overfitting", 
the fact that machine learning models tend to perform worse on new data than on their training data. 
Overfitting will be a central topic in chapter 3.

This concludes our very first example -- you just saw how we could build and a train a neural network to classify handwritten digits, in 
less than 20 lines of Python code. In the next chapter, we will go in detail over every moving piece we just previewed, and clarify what is really 
going on behind the scenes. You will learn about "tensors", the data-storing objects going into the network, about tensor operations, which 
layers are made of, and about gradient descent, which allows our network to learn from its training examples.